In [338]:
import pandas as pd
import json
from sqlalchemy import create_engine
import zipfile
import tarfile
import bz2
import sys
import os
import numpy as np


In [339]:
#### load lookup tables
df_city = pd.read_csv('./data/location_ref/city_multilingu_citycoord_countrycode_countrycoord.csv', index_col=0)
df_city.head()

,language,CITY_NAME,city_lat_3857,city_long_3857,city_lat_long_3857,population,COUNTRY_ALPHA2_CODE,country_lat_3857,country_long_3857,country_lat_long_3857
0,nn,trnava,48.37775,17.586211,"48.37775, 17.586211111",65382.0,SK,49.0,20.0,"49.0, 20.0"
1,pl,trnawa,48.37775,17.586211,"48.37775, 17.586211111",65382.0,SK,49.0,20.0,"49.0, 20.0"
2,ps,ترناوا,48.37775,17.586211,"48.37775, 17.586211111",65382.0,SK,49.0,20.0,"49.0, 20.0"
3,rue,тырнава,48.37775,17.586211,"48.37775, 17.586211111",65382.0,SK,49.0,20.0,"49.0, 20.0"
4,si,ට්ර්නාවා,48.37775,17.586211,"48.37775, 17.586211111",65382.0,SK,49.0,20.0,"49.0, 20.0"


In [340]:
#### load lookup tables
df_country = pd.read_csv('./data/location_ref/country_multilingu_countrycode_countrycoord.csv', index_col=0)
df_country.head()

,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_NAME,country_lat_3857,country_long_3857,country_lat_long_3857
0,AFRIKAANS,AD,andorra,42.558333,1.555278,"42.558333333, 1.555277777"
1,AFRIKAANS,AE,verenigde arabiese emirate,24.400000,54.300000,"24.4, 54.3"
2,AFRIKAANS,AF,afganistan,34.000000,66.000000,"34.0, 66.0"
3,AFRIKAANS,AG,antigua en barbuda,17.116667,-61.850000,"17.116666666, -61.85"
4,AFRIKAANS,AI,anguilla,18.227222,-63.048889,"18.227222222, -63.048888888"


In [341]:
df_country.shape

(13389, 6)

### As test data:
#### load only two files from every hour folder from the first day of the month

In [268]:
tweet_data = []

# import from one day
#next folder: stream-2020-06/06 2/02/02

for folder in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/")):
    if not folder.startswith('.'):
        print('load folder', folder)
        
        t = 0
        for file in sorted(os.listdir("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder)):
            
            t += 1
            if file.endswith('.bz2'):
                
                print(t)
                print('load file', file)

                with bz2.open("./data/archiveteam-twitter-stream-2020-06/06/01/" + folder + "/" + file, "rt") as bzinput:


                    for i, line in enumerate(bzinput):
                        tweet = json.loads(line)
                        tweet_data.append(tweet)

            #include break statement to test
            # load only 2 files from each hour folder 
                if t == 2: break


load folder 00
1
load file 29.json.bz2
2
load file 30.json.bz2
load folder 01
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 02
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 03
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 04
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 05
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 06
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 07
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 08
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 09
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 10
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 11
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 12
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 13
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 14
1
load file 00.json.bz2
2
load file 01.json.bz2
load folder 15
1
load file 00.json.bz2
2
load file 01.j

In [ ]:
len(tweet_data)

### Functions

#### from countryname or cityname in tweet to TO_countrycoords

In [358]:
def get_country_coords(country_or_city_name_multilingu):
    
    """Function to retrieve the coordinates of the geographical center of a country 
    from the country name or a cityname that is contained in a string (usecase: tweet). 
    Based on the lookup in two pandas dataframes.
    
    Parameters:
    ----------------------------
    country_or_city_name_multilingu: string
    
    CSV files needed for the coordinates lookup:
    ----------------------------
    df_city = './data/location_ref/city_multilingu_citycoord_countrycode_countrycoord.csv'
    df_country = './data/location_ref/country_multilingu_countrycode_countrycoord.csv'
    
    """
    
    # make input string lowercase
    search_string = country_or_city_name_multilingu.lower()
    
    # account for special cases UK, US and USA 
    if country_or_city_name_multilingu == 'UK':
        print('UK found')
        return df_country.loc[df_country['COUNTRY_NAME'] == 'united kingdom']['country_lat_long_3857'].values
    
    if country_or_city_name_multilingu == 'US' or search_string == 'USA':
        return df_country.loc[df_country['COUNTRY_NAME'] == 'united states']['country_lat_long_3857'].values

    # check for country name
    if not (df_country[df_country['COUNTRY_NAME'].isin([search_string])]).empty:
        print('country found')
        return df_country.loc[df_country['COUNTRY_NAME'] == search_string]['country_lat_long_3857'].values[0]

    # if country name not found, check for city name
    elif not (df_city[df_city['CITY_NAME'].str.contains(search_string)]).empty:
        print('cityname found')
        idx = df_city[df_city['CITY_NAME'].str.contains(search_string)]['population'].idxmax()
        return df_city.iloc[idx]['country_lat_long_3857']
    
    print('No coords found')
    return None

In [362]:
# test
str = 'somewhere in france'
get_country_coords(str)

No coords found


#### from cityname in user_loc to FROm_countrycoords

In [330]:
def get_user_loc_country_code():
    pass

In [331]:


def get_coordinates(bounding_coords, city_name, user_location) -> str:
    if bounding_coords:
        return bounding_coords
    elif city_name:
        # Lookup city name coordinates
        return '79.125472, 28.982011'
    elif user_location:
        # Lookup user location
        return '79.125472, 28.982011'
    elif country_code:
        # Lookup country code
        return get_from_country_coords(country_code)



### Write to postgres

In [332]:
# Declare postgres config
HOST = 'localhost'
USERNAME = 'postgres'
PORT = '5432'
DB = 'tweets_multilingu'
PASSWORD = 'postgres'

# Create a postgres connection and assign it to 'engine' variable
engine = create_engine(f'postgres://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [333]:
def extract_url(urls) -> str:
    
    """ Function to extract the url from the entities dict of the tweet json"""
    
    found_url = ''
    url_given = urls
    
    if url_given != None:
        for url in url_given:
            found_url = url.get('url')
            
    return found_url

In [407]:
user_locs = []
def load(data):
    """ Loads transformed tweeets into the Postgres database
    Parameters:
    ------------
    tweets: List of tweet dictionaries that were retrieved from the Twitter API """

    insert_query = """INSERT INTO tweets_test (
        tweet_id, 
        timestamp_ms,
        tweet, 
        url_in_tweet, 
        user_url, 
        user_location, 
        user_loc_country_code,
        user_followers_count, 
        user_verified, 
        user_statuses_count,
        user_id,
        user_created_at,

        lang,  
        from_coords,
        from_country_coords,
        to_country_coords,
        tweet_country_code)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    
    counter = 0 
    counter_loc = 0 
    counter_country_ment = 0 
    # loop through list of jsons 
    for j in data:
        # check for valid entries and if tweet json contains any location information 
        valid_entry = j.get('created_at')
        user_loc = j.get('user', {}).get('location')
        place_attr = j.get('place')
        user_url = j.get('user', {}).get('url')
        urls = j.get('entities', {}).get('urls')
        
        # conditions to still be included ! 
        follower_c = j.get('user', {}).get('followers_count')
        ver = j.get('user', {}).get('verified')
        
        counter += 1
        
        if (valid_entry and len(urls) != 0 and user_url) and (user_loc or place_attr) and (ver or follower_c > 10000):
            
            #print(ver, follower_c)

            counter_loc += 1 
            
            # loop through multilingual country list 
            for c_name_multi in df_country['COUNTRY_NAME']:
                
                
                # check if one of the country names exist in tweet text
                if c_name_multi in j['text']:
                    counter_country_ment += 1
                    print(c_name_multi)

                    # get user dict
                    user = j.get('user')
                    # ger user_location if exits
                    user_location = user.get('location')
                    print(user_location)
                    user_locs.append(user_location)
                            
                    # get place dict if exists and extract first coordinate of the bounding box
                    place = j.get('place')
                    country_code = ''
                    tweet_country_code = ''
                    bounding_coords = ''
                    city_name = ''
                    from_coords = ''
                    from_country_coords = ''

                    if place != None:
                        country_code = place.get('country_code')
                        tweet_country_code = place.get('country_code')
                        
#                         bounding_coords = ','.join(list(map(lambda n: str(n), place.get('bounding_box').get('coordinates')[0][0])))
                        bounding_coords = ''
                        
                        city_name = place.get('full_name')
                        from_coords = get_coordinates(bounding_coords, city_name, user_location)
#                         from_country_coords = get_from_country_coords(country_code)
                        from_country_coords = ''
                        
                    to_country_coords = get_to_country_coords(c_name_multi)
                    print(to_country_coords)
                    
                    user_loc_country_code = 'XY'
                    
                    # get url mentionend in tweet from entities dict
                    found_url = extract_url(j.get('entities').get('urls'))
                    

                    # insert values into postgres DB                             
                    engine.execute(insert_query, (
                        j['id'], #id
                        j['timestamp_ms'], #timestamp_ms
                        j['text'], #tweet
                        found_url, #url_in_tweet
                        user['url'], #user_url
                        user_location, #user_location
                        user_loc_country_code,
                        user['followers_count'], #user_followers_count
                        user['verified'], #user_verified
                        user['statuses_count'], #user_statuses_count
                        user['id'], #user_id
                        user['created_at'], #user_created_at
                        
                        j['lang'], #lang
                        from_coords, #from_coords
                        from_country_coords, #from_country_coords
                        to_country_coords, #to_country_coords
                        tweet_country_code #tweet_country_code
                        )
         
                    )
    
    
    print('scanned jsons :', counter, 
          '\n with some location information and user url given :', counter_loc,
          '\n with some location information, user url and country name mentioned :',counter_country_ment)
    
     

In [408]:
load(tweet_data)

france
Drôme, Ardèche
country found
 47.0, 2.0
mq
Drôme, Ardèche
country found
 14.666666666, -61.0
usa
Australia
country found
 39.828175, -98.5795
भारत
New Delhi, India
country found
 22.8, 83.0
tai
Gauteng
country found
 14.0, 101.0
nga
Gauteng
country found
 66.416666666, 94.25
irac
Nawalgarh, jhunjhunu rajasthan
country found
 33.0, 43.0
ベルギー
芝浜高校準軍事組織研究同好会
country found
 50.641111111, 4.668055555
fici
Barcelona
country found
 -18.0, 178.0
cad
Venezuela
country found
 15.466667, 19.4
fas
Catalunya
country found
 32.0, -6.0
gana
New Delhi, Delhi
country found
 8.0, -0.5
ngana
New Delhi, Delhi
country found
 8.0, -0.5
nga
New Delhi, Delhi
country found
 66.416666666, 94.25
colombia
Bogotá, D.C., Colombia
country found
 4.0, -74.0
cad
Bogotá, D.C., Colombia
country found
 15.466667, 19.4
日本
Osaka→Nara→Chiba(Tokyo)
country found
 35.0, 136.0
gini
New York, USA
country found
 10.0, -11.0
yt
Worldwide
country found
 -12.843055555, 45.138333333
oman
Boulogne-Billancourt
country found
 21

fici
Venezuela
country found
 -18.0, 178.0
cina
Venezuela
country found
 35.844722222, 103.451944444
oman
amy.remeikis@theguardian.com
country found
 21.0, 57.0
iemen
Argentina
country found
 15.5, 48.0
usa
Venezuela
country found
 39.828175, -98.5795
cad
Caracas, Venezuela
country found
 15.466667, 19.4
indi
Lausanne (Switzerland)
country found
 22.8, 83.0
usa
Maryland/DC
country found
 39.828175, -98.5795
nga
Indonesia
country found
 66.416666666, 94.25
yt
St. Joseph, Missouri
country found
 -12.843055555, 45.138333333
fas
Vancouver, BC
country found
 32.0, -6.0
palav
♡♡♡
country found
 7.466667, 134.55
liban
Washington, DC
country found
 33.833333, 35.766667
uruguay
Uruguay
country found
 -32.733333333, -56.65
urugua
Uruguay
country found
 -32.733333333, -56.65
jersey
United States 
country found
 49.19, -2.11
tai
Bethesda, MD
country found
 14.0, 101.0
oman
東京 渋谷区
country found
 21.0, 57.0
fas
Brasil
country found
 32.0, -6.0
タイ
日本 東京
country found
 14.0, 101.0
البرازيل
ملاعب كرة ا

In [ ]:
                    counter_country_ment += 1
                    
                    print(c_name_multi)
                    to_country_coords = df_country.loc[df_country['COUNTRY_NAME'] == c_name_multi]['country_lat_long_3857'].values[0]
                    print(to_country_coords)

                    
                    user_location = user.get('location')
                    print(user_location)
                    
                    # get user loc coords
                    from_country_coords = "XY"
                    
                   
                                
                    